In [1]:
library(Signac)
library(Seurat)
library(GenomeInfoDb)
library(EnsDb.Hsapiens.v86)
library(BSgenome.Hsapiens.UCSC.hg38)
library(ensembldb)
library(JASPAR2018)
library(TFBSTools)
library(patchwork)
library(universalmotif)
library(MotifDb)
library(TFBSTools)
library(patchwork)

suppressMessages(library(dplyr))
library('Matrix')
set.seed(1)

library(RColorBrewer)
library(ggplot2)
library(pheatmap)
library(cowplot)
library(reshape2)

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min


Loading required package: S4Vectors

Loading required package: stats4


Attaching package: ‘S4Vectors’


The

In [2]:
#### set enviroment
home = '~/farm/endometrium/data/scATAC/'
setwd(home)
outdir = paste0(home, 'seurat.output-S2D19/')
system(paste0('mkdir -p ', outdir, '/clusters'))
#### 


In [3]:
### Load data
so = readRDS(file = paste0(outdir, 'data/M3_scRNAseq_labelsTransfer.rds'))

# Construct a Motif object and add it to the dataset

In [4]:
DefaultAssay(so) <- 'peaks'

In [5]:
# Get a list of motif position frequency matrices from the JASPAR, HOCOMOCO, HOMER and SwissRegulon databases
pfm <- getMatrixSet(
  x = JASPAR2018,
  opts = list(species = 9606, all_versions = FALSE)
)
# add data from hocomoco A
tf_motif_hoco=subset(MotifDb, organism=='Hsapiens' & dataSource=='HOCOMOCOv11-core-A')
pfmHoco = convert_motifs(tf_motif_hoco, 'TFBSTools-PWMatrix')
pfmHoco_names =  sapply(pfmHoco, function(x) x@ID)
names(tf_motif_hoco) = pfmHoco_names
for (i in 1:length(pfmHoco) ){
    # Convert into a PCM
    x = t(pfmHoco[[i]]@profileMatrix) - apply(pfmHoco[[i]]@profileMatrix, 2, min) # normalize per column
    pfmHoco[[i]]@profileMatrix = round(t(x / apply(x, 1, max))*100) # mulitply x100 and round
    class(pfmHoco[[i]]@profileMatrix) <- 'integer'
    pfm[[pfmHoco_names[i]]] <- pfmHoco[[i]]
}
# add data from hocomoco B
tf_motif_hoco=subset(MotifDb, organism=='Hsapiens' & dataSource=='HOCOMOCOv11-core-B')
pfmHoco = convert_motifs(tf_motif_hoco, 'TFBSTools-PWMatrix')
pfmHoco_names =  sapply(pfmHoco, function(x) x@ID)
names(tf_motif_hoco) = pfmHoco_names
for (i in 1:length(pfmHoco) ){
    # Convert into a PCM
    x = t(pfmHoco[[i]]@profileMatrix) - apply(pfmHoco[[i]]@profileMatrix, 2, min) # normalize per column
    pfmHoco[[i]]@profileMatrix = round(t(x / apply(x, 1, max))*100) # mulitply x100 and round
    class(pfmHoco[[i]]@profileMatrix) <- 'integer'
    pfm[[pfmHoco_names[i]]] <- pfmHoco[[i]]
}
# add data from hocomoco C
tf_motif_hoco=subset(MotifDb, organism=='Hsapiens' & dataSource=='HOCOMOCOv11-core-C')
pfmHoco = convert_motifs(tf_motif_hoco, 'TFBSTools-PWMatrix')
pfmHoco_names =  sapply(pfmHoco, function(x) x@ID)
names(tf_motif_hoco) = pfmHoco_names
for (i in 1:length(pfmHoco) ){
    # Convert into a PCM
    x = t(pfmHoco[[i]]@profileMatrix) - apply(pfmHoco[[i]]@profileMatrix, 2, min) # normalize per column
    pfmHoco[[i]]@profileMatrix = round(t(x / apply(x, 1, max))*100) # mulitply x100 and round
    class(pfmHoco[[i]]@profileMatrix) <- 'integer'
    pfm[[pfmHoco_names[i]]] <- pfmHoco[[i]]
}
# add data from SwissRegulon                       
tf_motif_SwissRegulon=subset(MotifDb, organism=='Hsapiens' & dataSource=='SwissRegulon')
pfmSwissRegulon = convert_motifs(tf_motif_SwissRegulon, 'TFBSTools-PWMatrix')
pfmSwissRegulon_names =  sapply(pfmSwissRegulon, function(x) x@ID)
names(tf_motif_SwissRegulon) = pfmSwissRegulon_names
for (i in 1:length(pfmSwissRegulon) ){
    # Convert into a PCM
    x = t(pfmSwissRegulon[[i]]@profileMatrix) - apply(pfmSwissRegulon[[i]]@profileMatrix, 2, min) # normalize per column
    pfmSwissRegulon[[i]]@profileMatrix = round(t(x / apply(x, 1, max))*100) # mulitply x100 and round
    class(pfmSwissRegulon[[i]]@profileMatrix) <- 'integer'
    pfm[[pfmSwissRegulon_names[i]]] <- pfmSwissRegulon[[i]]
}
# add data from HOMER                       
tf_motif_HOMER=subset(MotifDb, dataSource=='HOMER')
pfmHOMER = convert_motifs(tf_motif_HOMER, 'TFBSTools-PWMatrix')
pfmHOMER_names =  sapply(pfmHOMER, function(x) x@ID)
names(pfmHOMER) = paste0( sapply( strsplit(pfmHOMER_names, '\\('), head, 1), '.HOMER')
pfmHOMER = pfmHOMER[ grep('[a-z]',  names(pfmHOMER), invert = T) ] # remove non-human (lowercase)         
for (i in 1:length(pfmHOMER) ){
    # Convert into a PCM
    x = t(pfmHOMER[[i]]@profileMatrix) - apply(pfmHOMER[[i]]@profileMatrix, 2, min) # normalize per column
    pfmHOMER[[i]]@profileMatrix = round(t(x / apply(x, 1, max))*100) # mulitply x100 and round
    class(pfmHOMER[[i]]@profileMatrix) <- 'integer'
    pfmHOMER[[i]]@name = gsub('.HOMER', '', names(pfmHOMER)[i])
    pfm[[names(pfmHOMER)[i]]] <- pfmHOMER[[i]]
}

In [6]:
# Add name in id
idx =  grep('^MA', names(pfm) )
for (i in idx ){
    pfm[[i]]@ID = paste0(pfm[[i]]@name, '.', pfm[[i]]@ID)
    names(pfm)[i] = pfm[[i]]@ID
    }

In [7]:
# remove pcm with NAs
pfm = pfm[ ! sapply(pfm, function(x) any(is.na(x@profileMatrix))) ]

In [8]:
length(pfm)
table(unlist(sapply(pfm, function(x) x@tags$dataSource)))

[1] 1681


HOCOMOCOv11-core-A HOCOMOCOv11-core-B HOCOMOCOv11-core-C              HOMER 
               181                 84                135                146 
      SwissRegulon 
               683 

In [ ]:
# Scan the DNA sequence of each peak for the presence of each motif
motif.matrix <- CreateMotifMatrix(
  features = StringToGRanges(rownames(so), sep = c(":", "-")),
  pwm = pfm,
  genome = 'hg38',
  sep = c(":", "-")
)
# Create a new Mofif object to store the results
motif <- CreateMotifObject(
  data = motif.matrix,
  pwm = pfm
)
# Add the Motif object to the assay
so[['peaks']] <- AddMotifObject(
  object = so[['peaks']],
  motif.object = motif
)

In [ ]:
# Compute  sequence characteristics of the peaks: GC content, sequence length, and dinucleotide frequency
so <- RegionStats(
  object = so,
  genome = BSgenome.Hsapiens.UCSC.hg38,
  sep = c(":", "-")
)

# Finding active TFmotifs - ChromVAR

This tends to give similar results as performing an enrichment test on differentially accessible peaks between the cell types

In [ ]:
so <- RunChromVAR(
  object = so,
  genome = BSgenome.Hsapiens.UCSC.hg38
)

In [ ]:
saveRDS(so,  paste0(outdir, '/data/M4_TFmotif_activities.rds'))

In [3]:
# so = readRDS(paste0(outdir, '/data/M4_TFmotif_activities.rds'))

In [ ]:
motif2names = lapply(pfm, function(x) x@name)
names(motif2names) = sapply(pfm, function(x) x@ID) %>% gsub('_', '-', .)

In [ ]:
DefaultAssay(so) <- 'chromvar'
for( cl in levels(Idents(so)) ){
  message('Testing ', cl,'\n')
  differential.activity = FindMarkers(so, ident.1 = cl, test.use = "LR", only.pos = TRUE, latent.vars = 'nCount_peaks', 
                                      logfc.threshold = 0.1, min.pct = 0.1)
  differential.activity$TF = motif2names[ rownames(differential.activity) ] %>% as.character(.)
  write.csv(differential.activity, file = paste0(outdir, '/clusters/', cl, '_differential.activity_LR.csv'))
}

# Compare Ciliated and new secretory vs PGR+

In [9]:
retrieve_top10 = function(differential.activity, n = 10, name = F){
    # Fix homer names
  idx = grep('HOMER', differential.activity$X)
  differential.activity$TF[ idx ] = gsub('.HOMER', '', differential.activity$X[ idx ])
  # Fix hocomoco synonyms
  differential.activity$TF2 = differential.activity$TF
  idx = differential.activity$TF %in% syn2gen$V1
  differential.activity$TF2[ idx ] =  syn2gen$V2[ match(differential.activity$TF[ idx ], syn2gen$V1) ] 
  differential.activity = differential.activity[ ! duplicated(differential.activity$TF2) , ]
  differential.activity = subset(differential.activity, avg_logFC > 0.5)
  differential.activity = subset(differential.activity, p_val_adj < 0.01)
    
  tfs = head(differential.activity$X, n = n) 
  if(name)
    tfs = head(differential.activity$TF2, n = n) 
  return(tfs)
}

In [10]:
# Load synonyms to annorate Tfs to their symbol
syn2gen = read.delim('~/secondary2primary_genename.annot', stringsAsFactors = F, header = F)
syn2gen = syn2gen[ order(syn2gen$V1 == syn2gen$V2, decreasing = T) , ]
syn2gen = syn2gen[ ! duplicated(syn2gen$V1) , ] # remove secondaries with several synonyms ( different from the primary )

In [ ]:
DefaultAssay(so) <- 'chromvar'
TOP10 = list()
All = list()

# PGR
differential.activity = FindMarkers(so, 
                                    ident.1 = c('inter_PGRpos'), ident.2 = c('new_secretory', 'ciliated'), 
                                    test.use = "LR", only.pos = TRUE, latent.vars = 'nCount_peaks', 
                                    logfc.threshold = 0.1, min.pct = 0.1)
differential.activity$TF = motif2names[ rownames(differential.activity) ] %>% as.character(.)
differential.activity$X = rownames(differential.activity)
write.csv(differential.activity, file = paste0(outdir, '/clusters/PGR_vs_other_differential.activity_LR.csv'))
TOP10[['PGR']] = retrieve_top10(differential.activity)
All[['PGR']] = retrieve_top10(differential.activity, n = 20000, name = T)

# ciliated
differential.activity = FindMarkers(so, 
                                    ident.1 = 'ciliated', ident.2 = c('inter_PGRpos', 'new_secretory'), 
                                    test.use = "LR", only.pos = TRUE, latent.vars = 'nCount_peaks', 
                                    logfc.threshold = 0.1, min.pct = 0.1)
differential.activity$TF = motif2names[ rownames(differential.activity) ] %>% as.character(.)
differential.activity$X = rownames(differential.activity)
write.csv(differential.activity, file = paste0(outdir, '/clusters/ciliated_vs_other_differential.activity_LR.csv'))
TOP10[['ciliated']] = retrieve_top10(differential.activity)
All[['ciliated']] = retrieve_top10(differential.activity, n = 20000, name = T)

# new_secretory
differential.activity = FindMarkers(so, 
                                    ident.1 = 'new_secretory', ident.2 = c('inter_PGRpos', 'ciliated'), 
                                    test.use = "LR", only.pos = TRUE, latent.vars = 'nCount_peaks', 
                                    logfc.threshold = 0.1, min.pct = 0.1)
differential.activity$TF = motif2names[ rownames(differential.activity) ] %>% as.character(.)
differential.activity$X = rownames(differential.activity)
write.csv(differential.activity, file = paste0(outdir, '/clusters/newsecretory_vs_other_differential.activity_LR.csv'))
TOP10[['new_secretory']] = retrieve_top10(differential.activity)
All[['new_secretory']] = retrieve_top10(differential.activity, n = 20000, name = T)

In [ ]:
soF = subset(so, cells = names(Idents(so))[ Idents(so) %in% c('inter_PGRpos', 'ciliated') ] )

In [ ]:
Idents(soF) = factor(Idents(soF), levels =  rev(c('inter_PGRpos', 'ciliated')))

In [ ]:
# DoHeatmap(soF, features = unlist(TOP10), slot = 'data') + scale_fill_gradientn(colors = c("grey", "red"), )
# ggsave(device = 'pdf', file = paste0(outdir, '/clusters/heatmap_TFmotifs_invivo.pdf'), width = 12, height = 10)
DotPlot(object = soF, features = rev(unlist(TOP10))) + theme(axis.text.x = element_text(angle = 45, vjust = 1, hjust = 1))
ggsave(device = 'pdf', file = paste0(outdir, '/clusters/dotplot_TFmotifs_invivo.pdf'), width = 12, height = 5)

In [ ]:
TFmotifs = c('E2F4-HUMAN.H11MO.0.A', "E2F4.MA0470.1", "E2F4.HOMER", 'PGR.HOMER','ESR1.MA0112.3', 
            'TP73.SwissRegulon', 'FOXJ1.SwissRegulon', "RFX2-HUMAN.H11MO.0.A", 'RFX2.MA0600.2', 
            'HNF1B-HUMAN.H11MO.0.A', 'CEBPB-HUMAN.H11MO.0.A', "CEBPB.MA0466.2", "CEBPB.SwissRegulon")
# DoHeatmap(soF, features = TFmotifs, slot = 'data') + scale_fill_gradientn(colors = c("white", "red"), )
# ggsave(device = 'pdf', file = paste0(outdir, '/clusters/heatmap_TFmotifs_invivo_intersectRNA.pdf'), width = 12, height = 10)
DotPlot(object = soF, features = rev(TFmotifs)) + 
# theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 1)) + 
theme(axis.text.x = element_text(angle = 45, vjust = 1, hjust = 1))
ggsave(device = 'pdf', file = paste0(outdir, '/clusters/dotplot_TFmotifs_invivo_intersectRNA.pdf'), width = 7, height = 5.2)

## Plot top TF motifs

In [11]:
DefaultAssay(so) <- 'chromvar'
TOP5 = list()
All = list()
for( cl in rev(levels(Idents(so))) ){
  message('Testing ', cl,'\n')
  differential.activity = read.csv(file = paste0(outdir, '/clusters/', cl, '_differential.activity_LR.csv'), stringsAsFactors = F)
  # Fix homer names
  idx = grep('HOMER', differential.activity$X)
  differential.activity$TF[ idx ] = gsub('.HOMER', '', differential.activity$X[ idx ])
  # Fix hocomoco synonyms
  differential.activity$TF2 = differential.activity$TF
  idx = differential.activity$TF %in% syn2gen$V1
  differential.activity$TF2[ idx ] =  syn2gen$V2[ match(differential.activity$TF[ idx ], syn2gen$V1) ] 
  differential.activity = differential.activity[ ! duplicated(differential.activity$TF2) , ]
    
  TOP5[[cl]] = head(differential.activity$X, n = 6) 
  All[[cl]] = subset(differential.activity, avg_logFC > 1 & p_val_adj < 0.01 )$TF2
  if( length(TOP5[[cl]]) > 1 )
      for ( tf in TOP5[[cl]] ) {
          # Get MotifPlot
          p2 = MotifPlot(
            object = so,
            motifs = gsub('-', '_', tf),
            assay = 'peaks'
          )
          p1 = FeaturePlot(
            object = so,
            features = tf,
            pt.size = 0.1,
            cols = c("grey", "red"), min.cutoff = 0.5)
          P = plot_grid(plotlist = list(p1, p2), ncol = 1)
          save_plot(plot = P, filename = paste0(outdir, '/clusters/', cl, '_differential.activity_motifPlot_',tf,'.pdf'), base_width = 4, base_height = 7)
       }
}

All = lapply(All, unique)
duplicatedTFs = unlist( All)[ duplicated(unlist( All)) ]
All_clusters_specific = lapply(All, setdiff, duplicatedTFs )
All_clusters_specific

Testing 11


Testing 10


Testing 9


Testing 8


Testing 7


Testing 6


Testing 5


Testing 4


Testing 3


Testing 2


Testing 1


Testing 0




$`11`
character(0)

$`10`
 [1] "FOXA2"     "DBP"       "RFX"       "RFX1"      "FOXJ2"     "RFX7"     
 [7] "SNAI2"     "TCF12"     "TFE2"      "FOXH1"     "REST"      "ONECUT2"  
[13] "NHLH1"     "BRCA1"     "BHA15"     "MESP1"     "ESRRA"     "MYF6"     
[19] "MSC"       "OLIG2"     "GLI1"      "ZIC1"      "ZKSC1"     "MYB"      
[25] "REST-NRSF" "MTF1"      "ZIC4"      "ZSCAN4"    "ZIC3"      "TBX21"    
[31] "ESRRB"     "NFIA"      "CTCFL"    

$`9`
 [1] "FOXJ1"        "PRDM6"        "IRF1"         "DMRT1"        "ZN770"       
 [6] "IRF9"         "FOXJ3"        "FOXA1:AR"     "STAT1::STAT2" "FUBP1"       
[11] "LEF1"         "BPTF"         "SMCA5"        "ZN121"        "IRF3"        
[16] "IKZF2"        "HNF4G"        "IRF7"         "NFATC2"       "HNF4A"       
[21] "NFATC3"       "TCF3"         "RBPJ"         "NFKB1"        "NR2F1"       
[26] "RXRG"         "RXRB"         "COUP-TFII"    "TF65"         "NR1H2::RXRA" 

$`8`
character(0)

$`7`
 [1] "ZNF740"     "ZNF281"     "HOXB5"      "ZN281"      "ZNF263"    
 [6] "RXRA"       "EWSR1-FLI1" "ZBT17"      "SP4"        "E2F7"      
[11] "KLF15"      "PPARG"      "E2F1"       "WRNIP1"     "ZSC22"     
[16] "E2F6"       "PTF1A"      "NKX25"      "ETV5"       "ZN263"     
[21] "GTF2I"      "ZN467"      "MAZ"        "VEZF1"      "SPIB"      
[26] "ZN341"      "RARA"       "PATZ1"      "ETS2"       "E2F3"      
[31] "SUH"        "TBX3"       "INSM1"      "NFIC"       "MTA3"      
[36] "KLF12"      "ZN257"      "LYL1"       "ZN816"      "TFAP2A"    
[41] "ZN320"      "AP2A"       "ZN436"      "ZNF467"     "SRBP2"     
[46] "THAP1"      "GATA2"      "SNAI1"      "NR1I3"      "ZN563"     
[51] "SMAD1"      "TLX1"      

$`6`
 [1] "ELF5"               "ELF3"               "EHF"               
 [4] "GRHL2"              "ELF1"               "KLF5"              
 [7] "KLF4"               "ELF4"               "ETV6"              
[10] "ERG"                "SPDEF"              "KLF1"              
[13] "ELF2"               "ETV2"               "ELK4"              
[16] "GABPA"              "ETV7"               "KLF9"              
[19] "ETS1"               "NF-E2"              "BACH2"             
[22] "GRHL1"              "SPIC"               "ZBTB7A"            
[25] "PBX3"               "JUN"                "FOSL1::JUND(var.2)"
[28] "ATF4"               "TFCP2"              "CEBPA"             
[31] "CEBPB"              "IRF8"               "CEBPE"             
[34] "NFIL3"              "IRF4"               "DDIT3"             
[37] "CEBP:AP1"          

$`5`
[1] "CTCF"  "NFYB"  "RAD21" "SMC3" 

$`4`
 [1] "TEAD2" "TEAD4" "TEAD1" "ISL1"  "TEAD"  "TEAD3" "HOXB3" "ZFHX3" "ESX1" 
[10] "HOXD3" "PBX2"  "HESX1" "GBX1"  "HXB7"  "HOXB7" "HXA9" 

$`3`
[1] "BACH1" "PAX6"  "PAX9"  "RELA" 

$`2`
 [1] "TP53"      "TP63"      "TP73"      "BCL11A"    "RUNX3"     "TFCP2L1"  
 [7] "AHR"       "ARNT2"     "EGR3"      "MAF::NFE2" "DUXA"      "TCF7L2"   

$`1`
character(0)

$`0`
 [1] "SPI1"        "HOXA4"       "PITX1"       "PITX2"       "PAX5"       
 [6] "POU2F3"      "HOXA13"      "ONECUT3"     "HNF1A"       "ZNF384"     
[11] "RCOR1"       "TFAP2D"      "NFE2"        "FOSB"        "BATF::JUN"  
[16] "SP8"         "SMARCC1"     "FOS::JUND"   "FOSL1::JUN"  "FOS::JUN"   
[21] "FOSL2::JUN"  "FOSL1::JUND" "FOSB::JUNB"  "FOSL2::JUND" "JUN::JUNB"  
[26] "FOSL1::JUNB" "FOSL2::JUNB" "FOS::JUNB"   "E2F4"        "MBD2"       
[31] "NRF1"        "HIC1"

In [ ]:
# DoHeatmap(so, features = unlist(TOP5), slot = 'data') + scale_fill_gradientn(colors = c("grey", "red"))
# ggsave(device = 'pdf', file = paste0(outdir, '/general/heatmap_TFmotifs.pdf'), width = 12, height = 10)

In [12]:
TF2plot = list('1' = c('E2F4-HUMAN.H11MO.0.A', "E2F4.MA0470.1", "E2F4.HOMER", 'PGR.HOMER','ESR1.MA0112.3'), 
             '10' = c('TP73.SwissRegulon', 'FOXJ1.SwissRegulon', "RFX2-HUMAN.H11MO.0.A", 'RFX2.MA0600.2'))


for( cl in names(TF2plot) ){
  message('Testing ', cl,'\n')
  differential.activity = read.csv(file = paste0(outdir, '/clusters/', cl, '_differential.activity_LR.csv'), stringsAsFactors = F)
  rownames(differential.activity) = differential.activity$X
  for ( tf in TF2plot[[cl]]) {
      # Get MotifPlot
      p2 = MotifPlot(
        object = so,
        motifs = gsub('-', '_', tf),
        assay = 'peaks'
      )
      p1 = FeaturePlot(
        object = so,
        features = tf,
        pt.size = 0.2,
        cols = c("grey", "red"), min.cutoff = 0.5)
      P = plot_grid(plotlist = list(p1, p2), ncol = 1)
     save_plot(plot = P, filename = paste0(outdir, '/clusters/', cl, '_differential.activity_motifPlot_',tf,'.pdf'), base_width = 4, base_height = 7)
   }
}

Testing 1


Testing 10




# Finding overrepresented TF motifs in DARs

Search for DNA motifs that are overrepresented in a set of peaks that are differentially accessible between cell clusters
that we previously computed

In [ ]:
# THERE IS a BUG IN THIS CODE WHEN ADDING MOTIFS APPART from JASPAR
# DefaultAssay(so) <- 'peaks'
# for( cl in rev(levels(Idents(so))) ){
#   cat('Loading ', cl,' DARs\n')
#   da_peaks2 = read.csv(file = paste0(outdir, '/clusters/DARs/', cl, '_LR.csv'), stringsAsFactors = F)
#   da_peaks2 = subset(da_peaks2, avg_logFC > 0 & p_val < 0.0001 )
#   rownames(da_peaks2) = da_peaks2$X
#   enriched.motifs <- FindMotifs(object = so,
#     features = rownames(da_peaks2)
#   )
# #   enriched.motifs = enriched.motifs[ order(enriched.motifs$pvalue, decreasing = F) ,]
#   enriched.motifs$cluster = cl
# #   # patch 
# #   enriched.motifs$true_pvalue =   as.numeric(enriched.motifs[ 1 , grep('^pvalue', names(enriched.motifs)) ])
# #   enriched.motifs = enriched.motifs[  , - grep('^pvalue', names(enriched.motifs)) ]
# #   enriched.motifs = enriched.motifs[order(enriched.motifs$true_pvalue, decreasing = F), ]
#   write.csv(enriched.motifs, file = paste0(outdir, '/clusters/DARs/', cl, '_enrichedTFmotifs.csv'))
# }

In [ ]:
sessionInfo()